# Scraping kost data from olx.com

## Preparing some libraries

In [1]:
import re
import os
import codecs
import time
import pyprind
import pandas as pd
import numpy as np

import requests 
from bs4 import BeautifulSoup

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

os.environ['WDM_SSL_VERIFY']='0'

## Get all url from specific locations

In [2]:
link0  = 'https://www.olx.co.id/indekos_c4833?sorting=desc-creation'
link1  = 'https://www.olx.co.id/jakarta-selatan_g4000030/indekos_c4833'
link2  = 'https://www.olx.co.id/jakarta-barat_g4000028/indekos_c4833'
link3  = 'https://www.olx.co.id/jakarta-pusat_g4000029/indekos_c4833'
link4  = 'https://www.olx.co.id/jakarta-timur_g4000031/indekos_c4833'
link5  = 'https://www.olx.co.id/jakarta-utara_g4000032/indekos_c4833'
link6  = 'https://www.olx.co.id/surabaya-kota_g4000216/indekos_c4833'
link7  = 'https://www.olx.co.id/malang-kota_g4000212/indekos_c4833'
link8  = 'https://www.olx.co.id/sidoarjo-kab_g4000202/indekos_c4833'
link9  = 'https://www.olx.co.id/malang-kab_g4000192/indekos_c4833'
link10 = 'https://www.olx.co.id/bandung-kota_g4000018/indekos_c4833'
link11 = 'https://www.olx.co.id/bekasi-kota_g4000020/indekos_c4833'
link12 = 'https://www.olx.co.id/depok-kota_g4000024/indekos_c4833'
link13 = 'https://www.olx.co.id/bekasi-kab_g4000003/indekos_c4833'
link14 = 'https://www.olx.co.id/bogor-kab_g4000004/indekos_c4833'
link15 = 'https://www.olx.co.id/bogor-kota_g4000021/indekos_c4833'
link16 = 'https://www.olx.co.id/tangerang-selatan-kota_g4000080/indekos_c4833'
link17 = 'https://www.olx.co.id/tangerang-kota_g4000079/indekos_c4833'
link18 = 'https://www.olx.co.id/tangerang-kab_g4000076/indekos_c4833'
link19 = 'https://www.olx.co.id/sleman-kab_g4000071/indekos_c4833'
link20 = 'https://www.olx.co.id/yogyakarta-kota_g4000072/indekos_c4833'
link21 = 'https://www.olx.co.id/bantul-kab_g4000068/indekos_c4833'
link22 = 'https://www.olx.co.id/semarang-kota_g4000065/indekos_c4833'
link23 = 'https://www.olx.co.id/surakarta-kota_g4000066/indekos_c4833'
link24 = 'https://www.olx.co.id/denpasar-kota_g4000225/indekos_c4833'
link25 = 'https://www.olx.co.id/kab-badung_g4000217/indekos_c4833'
link26 = 'https://www.olx.co.id/medan-kota_g4000131/indekos_c4833'
link27 = 'https://www.olx.co.id/bandar-lampung-kota_g4000382/indekos_c4833'
link28 = 'https://www.olx.co.id/palembang-kota_g4000368/indekos_c4833'
link29 = 'https://www.olx.co.id/kalimantan-selatan_g2000013/indekos_c4833'
link30 = 'https://www.olx.co.id/aceh-di_g2000001/indekos_c4833'
link31 = 'https://www.olx.co.id/kalimantan-timur_g2000015/indekos_c4833'

In [ ]:
# get all url using sellenium
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link31)

kos_urls = []
ads = 3000 # total kost ads u want to scrap, or u can found from total ads that the web said
max_loop = int(ads / 20)

# load all ads
for val in range(max_loop):
    load_more = driver.find_element_by_xpath("//span[.='muat lainnya']")
    time.sleep(1)
    load_more.click()
    time.sleep(3)

In [54]:
# get link using beautifulsoup
html = BeautifulSoup(driver.page_source, 'html.parser')
all_links = html.find_all('li', class_='EIR5N')
# len(all_links)
for link in all_links:
    kos_urls.append('https://www.olx.co.id'+ link.a['href'])


In [55]:
# check if there is any duplicate links
print(len(set(all_links)), len(all_links))
driver.close()

152 152


In [56]:
# write it to file txt
kos_url_file = open('olx_urls31.txt',  'w',  encoding="utf-8")
for url in kos_urls:
    kos_url_file.write('{}\n'.format(url))

In [8]:
links = list(set(kos_urls)) # add non duplicate links to files

In [16]:
kos_url_file = open('.\olx\olx_urls.txt',  'w',  encoding="utf-8") # all links from link1 to link31
for url in links:
    kos_url_file.write('{}\n'.format(url))

## Load urls

In [2]:
# there is duplicate links
def load_multiple_links():
    links = []

    max = 31

    for i in range(1, max+1):
        urls = open('olx_urls{}.txt'.format(i),  'r',  encoding="utf-8")

        for url in urls:
            links.append(url.replace('\n', ''))
        
    print(len(links), len(set(links)))
    return links

# no duplicate links
def load_links():
    links = []
    urls = open('.\olx\olx_urls.txt',  'r',  encoding="utf-8")
    for url in urls:
        links.append(url.replace('\n', ''))
        
    print(len(links), len(set(links)))
    return links

In [3]:
links = load_links()

11202 11202


In [5]:
# batching
url_batch_1 = links[:1438]
url_batch_2 = links[1438:2500]
url_batch_3 = links[2500:3500]
url_batch_4 = links[3500:4500]
url_batch_5 = links[4500:5500]
url_batch_6 = links[5500:6500]
url_batch_7 = links[6500:7500]
url_batch_8 = links[7500:8500]
url_batch_9 = links[8500:9500]
url_batch_10 = links[9500:10500]
url_batch_11 = links[10500:]

print(len(url_batch_1) + len(url_batch_2) + len(url_batch_3) + len(url_batch_4) + len(url_batch_5) + len(url_batch_6) + len(url_batch_7) + len(url_batch_8) + len(url_batch_9) + len(url_batch_10) + len(url_batch_11))
print(len(url_batch_1) , len(url_batch_2) , len(url_batch_3) , len(url_batch_4) , len(url_batch_5) , len(url_batch_6) , len(url_batch_7) , len(url_batch_8) , len(url_batch_9) , len(url_batch_10) , len(url_batch_11))

In [4]:
# 6500 - 10500
url_batch_7 = links[6500:7900]
url_batch_8 = links[7900:9300]
url_batch_9 = links[9300:10500]

print(len(url_batch_7) + len(url_batch_8) + len(url_batch_9))
print(len(url_batch_7) , len(url_batch_8) , len(url_batch_9))

4000
1400 1400 1200


## Declaring function for scraping kost data

In [3]:
def scraping_multiple_links(links):
    titles, owners, locations, phones = [], [], [], []
    # login 
    login_url = 'https://www.olx.co.id/'
    login_data = {
        'email': 'your email',
        'password': 'your password'
    }
    chrome_driver_path = 'C:\Program Files (x86)\chromedriver.exe'

    # login 
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(login_url)
    log_in = driver.find_element_by_xpath('//span[contains(text(), "Login")]')
    log_in.click()
    time.sleep(2)
    login_by_email = driver.find_element_by_xpath("//span[.='Login/Daftar dengan Email']")
    login_by_email.click()
    time.sleep(3)
    email_field = driver.find_element_by_name('email')
    email_field.send_keys(login_data['email'])
    email_field.submit()
    time.sleep(5)
    pswd_field = driver.find_element_by_name('password')
    pswd_field.send_keys(login_data['password'])
    pswd_field.submit()
    time.sleep(3)
    

    bar = pyprind.ProgBar(len(links), track_time=True, title='Scraping kos data from olx')

    # try get dta each link
    for link in links:
        try:
            # login again if logged out in the middle of process
            if len(driver.find_element_by_xpath('//span[contains(text(), "Login")]').text) > 0:
                log_in = driver.find_element_by_xpath('//span[contains(text(), "Login")]')
                log_in.click()
                time.sleep(2)
                login_by_email = driver.find_element_by_xpath("//span[.='Login/Daftar dengan Email']")
                login_by_email.click()
                time.sleep(3)
                email_field = driver.find_element_by_name('email')
                email_field.send_keys(login_data['email'])
                email_field.submit()
                time.sleep(5)
                pswd_field = driver.find_element_by_name('password')
                pswd_field.send_keys(login_data['password'])
                pswd_field.submit()
                time.sleep(3)
        except:
            pass
        
        driver.get(link)
        try:
            # click show number to make page show it
            show_number = driver.find_element_by_xpath('//div[contains(text(), "Tampilkan nomor")]')
            parent_class = show_number.find_element_by_xpath("./..").get_attribute('class')
            show_number.click()
            time.sleep(3)
        except:
            pass
        
        
        page = BeautifulSoup(driver.page_source, 'html.parser') # load to beautifulsoup type
        # get data, if null then '-'
        try:        
            title = page.find('h1', class_='_3rJ6e').text
        except:
            title = '-'
        
        try:
            location = page.find('span', class_='_2FRXm').text
        except:
            location = '-'
            
        try:
            owner = page.find('div', class_='_3oOe9').text
        except:
            owner = '-'
            
        try:
            phone = page.find('div', class_='_1uXYV').text
        except:
            phone = '-'
            
        # save to list
        titles.append(title)
        owners.append(owner)
        locations.append(location)
        phones.append(phone)
        delays = [2, 3, 4]
        delay = np.random.choice(delays)
        time.sleep(delay)
        bar.update()
        
    driver.close()   
    return titles, locations, owners, phones

def scraping_single_link(link):
    # titles, owners, locations, phones = [], [], [], []
    login_url = 'https://www.olx.co.id/'
    login_data = {
        'email': 'your email',
        'password': 'your password'
    }
    chrome_driver_path = 'C:\Program Files (x86)\chromedriver.exe'

    # login 
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(login_url)
    log_in = driver.find_element_by_xpath('//span[contains(text(), "Login")]')
    log_in.click()
    time.sleep(2)
    login_by_email = driver.find_element_by_xpath("//span[.='Login/Daftar dengan Email']")
    login_by_email.click()
    time.sleep(3)
    email_field = driver.find_element_by_name('email')
    email_field.send_keys(login_data['email'])
    email_field.submit()
    time.sleep(5)
    pswd_field = driver.find_element_by_name('password')
    pswd_field.send_keys(login_data['password'])
    pswd_field.submit()
    time.sleep(3)
    
    driver.get(link)
    try:
        show_number = driver.find_element_by_xpath('//div[contains(text(), "Tampilkan nomor")]')
        parent_class = show_number.find_element_by_xpath("./..").get_attribute('class')
        show_number.click()
        time.sleep(2)
    except:
        pass
    
    page = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:        
        title = page.find('h1', class_='_3rJ6e').text
    except:
        title = '-'
    
    try:
        location = page.find('span', class_='_2FRXm').text
    except:
        location = '-'
        
    try:
        owner = page.find('div', class_='_3oOe9').text
    except:
        owner = '-'
        
    try:
        phone = page.find('div', class_='_1uXYV').text
    except:
        phone = '-'
        
    driver.close() 
    return title, location, owner, phone

## Scraping kost data

In [5]:
links = url_batch_9[-71:] # get from batch link
titles, owners, locations, phones = [], [], [], []

In [7]:
# for login function
def login_olx(login_data, driver):
    log_in = driver.find_element(By.XPATH, '//span[contains(text(), "Login")]')
    log_in.click()
    time.sleep(2)
    login_by_email = driver.find_element(By.XPATH, "//span[.='Login/Daftar dengan Email']")
    login_by_email.click()
    time.sleep(3)
    email_field = driver.find_element(By.NAME, 'email')
    email_field.send_keys(login_data['email'])
    email_field.submit()
    time.sleep(3)
    pswd_field = driver.find_element(By.NAME, 'password')
    pswd_field.send_keys(login_data['password'])
    pswd_field.submit()
    time.sleep(3)

In [8]:
# without function that already declared

chrome_driver_path = 'C:\Program Files (x86)\chromedriver.exe'

# login 
login_url = 'https://www.olx.co.id/'
login_data = {
    'email': 'your email',
    'password': 'your password'
}

# login 
driver = webdriver.Chrome(service=Service(chrome_driver_path))
driver.get(login_url)
time.sleep(1)
login_olx(login_data, driver)

bar = pyprind.ProgBar(len(links), track_time=True, title='Scraping kos data from olx')

for link in links:
    try:
        if len(driver.find_element(By.XPATH, '//span[contains(text(), "Login")]').text) > 0:
            time.sleep(1)
            login_olx(login_data, driver)
    except:
        pass
    
    try:
        driver.get(link)
        try:
            if len(driver.find_element(By.XPATH, '//span[contains(text(), "Ups...")]').text) > 0:
                title = '-'
                location = '-'
                owner = '-'
                phone = '-'
        except:
            elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME , "_2b6xv")))
            show_number = driver.find_element(By.XPATH, '//div[contains(text(), "Tampilkan nomor")]')

            parent_class = show_number.find_element(By.XPATH, "./..").get_attribute('class')
            show_number.click()

            time.sleep(3)
            try:
                page = BeautifulSoup(driver.page_source, 'html.parser')
                
                try:        
                    title = page.find('h1', class_='_3rJ6e').text
                except:
                    title = '-'
                
                try:
                    location = page.find('span', class_='_2FRXm').text
                except:
                    location = '-'
                    
                try:
                    owner = page.find('div', class_='_3oOe9').text
                except:
                    owner = '-'
                    
                try:
                    phone = page.find('div', class_='_1uXYV').text
                except:
                    phone = '-'
            except:
                title = '-'
                location = '-'
                owner = '-'
                phone = '-'
    except:
        title = '-'
        location = '-'
        owner = '-'
        phone = '-'
    
    titles.append(title)
    owners.append(owner)
    locations.append(location)
    phones.append(phone)
    delays = [1, 2]
    delay = np.random.choice(delays)
    time.sleep(delay)
    bar.update()
    
driver.close()   

Scraping kos data from olx
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:14:47


In [9]:
# check duplicate data
print(len(links), len(set(titles)), len(set(locations)), len(set(owners)), len(set(phones)))
print(len(links), len(titles), len(locations), len(owners), len(phones))

71 61 50 59 58
71 71 71 71 71


## Saving data to excel

In [10]:

import pandas as pd
type = 'file_olx_'
batch = '9_2'

output_path = '.\olx\{}_{}.xlsx'.format(type, batch)
pd.DataFrame({'Judul':titles, 'Lokasi':locations, 'Owner Kos':owners, 'No Hp':phones, 'URL Kos':links}).to_excel(output_path, index=False, engine='xlsxwriter')

In [7]:
import pandas as pd
excl_list = []

for i in range(1, 10+1):
    if i != 9:
        excl_list.append(pd.read_excel('.\olx\\file_olx_{}.xlsx'.format(i)))
    else:
        excl_list.append(pd.read_excel('.\olx\\file_olx_{}.xlsx'.format(i))[:-71])

excl_merged = pd.DataFrame()

for excl_file in excl_list:
    excl_merged = excl_merged.append(
        excl_file, ignore_index=True)

excl_merged.to_excel('.\olx\\file_olx.xlsx', index=False)

In [8]:
import pandas as pd
df = pd.read_excel('.\olx\\file_olx.xlsx')

In [18]:
print(len(df['URL Kos'].unique()))
print(len(df['No Hp'].unique()))

11203
6472
